# Atelier Python Session 4

Maintenant que nous avons vu les bases et quelques opérations utils comme la lecture et l'ouverture de fichiers, nous allons désormais passer à des choses un peu plus amustantes! Aujourd'hui nous allons voir le **webscraping**!

Le [webscraping](https://fr.wikipedia.org/wiki/Web_scraping) nous permet d'aller parcourir le web avec notre script python, et interagir avec ce contu de différentes manières. Il existe plusieurs approches - nous verrons d'abord le webscraping de base à partir de requêtes http, puis comment interagir avec un [API](https://en.wikipedia.org/wiki/API), comme celui de [Nakala](https://www.nakala.fr/).

## 1. Webscraping avec Requests

D'abord, voyons un package qui nous sera très important: `requests`. Il s'agit encore d'un package qui fait partie de la distribution de base de python. Commencons par l'importer:

In [ ]:
import requests

Avec requests, nous avons accès à une fonction qui s'appelle `get()`. Avec cette fonction on pourra donne comme argument l'url d'un site web, pour aller le "visiter" comme si on allait sur internet dans un navigateur (pour que cela puisse fonctionner, il faudrait que votre ordinateur soit connect à internet!).

Faisons un essai avec wikipedia. La page d'acceuil de wikipedia se trouve à l'adresse suivant: [https://fr.wikipedia.org/wiki/Wikip%C3%A9dia:Accueil_principal](https://fr.wikipedia.org/wiki/Wikip%C3%A9dia:Accueil_principal). Nous allons donner cet url à `requests.get()`:

In [ ]:
url = "https://fr.wikipedia.org/wiki/Wikip%C3%A9dia:Accueil_principal"
response = requests.get(url)

print(response)

La fonction nous retourne l'instance d'une classe. On reviendra bientôt sur les classes, mais sachez qui cette instance contient des **proprietés** qui nous donnent plus d'informations sur cette requete que nous avons fait à internet. D'abord, il y a le `status_code`. Il s'agit d'un nombre codifié qui nous permet de savoir si notre requête a eu du succès.

- 200: la requête a été répondu avec succès.
- 404: requests n'a pas trouvé de réponse avec cet url.

In [ ]:
print(response.status_code)

Notre réponse aura d'autres proprietés qui seront remplis selon la nature de notre requête. La propieté `content` s'agit du contenu de notre page web en **bytes**:

In [ ]:
print(response.content)

La propriété `text` s'agit du contenu html de la page web sous forme de string:

In [ ]:
print(response.text)

Voila! Nous avons été chercher du contenu en ligne avec succès directement à partir de notre script python. En revanche, les réponses que nous avons eu pour l'instant ne sont pas d'une très grande utilité. Comment pouvons-nous parcourir ces données de manière intelligente?

## 2. BeautifulSoup

Pour prendre cette réponse et pour pouvoir trouver ce qu'il nous faut là dedans, nous allons utiliser un package qui s'appelle `BeautifulSoup`. Ce package ne fait pas partie de la distribution de python de base, donc il va falloir l'installer dans notre environment avant de pouvoir l'utiliser.

Nous verrons les environments de python et les **kernels** plus tard, mais pour l'instant, sachez que nous pouvons installer des packages directement dans l'environment colab en cours. Ces packages seront installé seulement pour ce script, donc il faudrait les réinstaller plus tard si vous voulez les utiliser à nouveau dans un autre notebook.

Dans colab, pour installer un package, on écrit `!pip install`, puis on donne le nom du package, comme ceci:

In [ ]:
!pip install beautifulsoup4

Mainenant qu'il est installé, nous pouvons l'importer. Voyons une nouvelle manière d'importer les packages qui nous permettre de clarifier un peu notre code. Pour l'instant, nous avons importé des packages en entier juste en utilisant `import`. Il est possible d'importer seulement des parties d'un pack ave cle mot clé `from`.

Ici, nous importons la classe `BeautifulSoup` du package qui s'appelle `bs4`:

In [ ]:
from bs4 import BeautifulSoup

Cette classe va désormais nous permettre de parcourir le contenu d'une requête http de manière plus intelligente. Prenons un exemple simple, nous allons utiliser `Beautifulsoup` pour trouver un element `<h1>` dans notre page web wikipedia.

D'abord, on crée une instance de la classe `BeautifulSoup`, en lui donnant deux arguments:
- Le contenu de la réponse en bytes (`response.content`)
- Un string qui dit à Beautiful Soup de quel type il s'agit (ici `html.parser`, on peut par exemple aussi donner du contenu xml à la place).

In [ ]:
bs = BeautifulSoup(response.content, 'html.parser')

Cette instance va me donner accès à un certain nombre de méthodes qui me permettront de parcourir et de trouver des choses dans le contenu html. Par exemple, je peux trouver tous les eleents `<h1>` avec la méthode `find_all()`:

In [ ]:
headings = bs.find_all("h1")

print(headings)

Remarquez que nous avons donc une liste de tous les elements `<h1>` ainsi que leurs proprietés et leur contenu. On peut accèder à ces elements directement de la manière suivant:

In [ ]:
first_heading = headings[0]

print(first_heading.get("id"))    # L'id de l'element html
print(first_heading.get("class")) # La ou les classes de l'element html
print(first_heading.string)       # Le contenu de l'élement html

### Wikipedia scraper

En assemblant ces outils, nous pouvons faire des choses intéressantes. Par exemple, wikipedia a un url qui permet de trouver un article aléatoire: [https://fr.wikipedia.org/wiki/Special:Random](https://fr.wikipedia.org/wiki/Special:Random). On sait que toutes les pages de wikipédia sont construites de la même manière, donc on pourra toujours retrouver par exemple le titre de cet article de la manière suivante:

In [ ]:
# Ma requete http initiale:
response = requests.get("https://fr.wikipedia.org/wiki/Special:Random")

# Je vérifie qu'il a été repondu avec succès:
if response.status_code == 200:
  print(response.url) # Comme il s'agit d'une redirection, j'imprime l'url final
  bs = BeautifulSoup(response.content, 'html.parser') # Je crée une instance de beautiful soup
  heading = bs.find(id = "firstHeading") # L'id du titre sur un page wikipédia a toujours l'id 'firstHeading'.
  print(heading.string) # J'imprime le contenu du titre:

Une autre fonctionalité utilie pourra être de télécharger une image provenant d'internet. Ecrivons une fonction qui fait ca. Pour ce faire on va utiliser un autre package qui rappelle `urllib` qui nous permettra d'ouvrir une image stocké à un url directement:

In [ ]:
import urllib

def download_image(url, outpath):
  img = urllib.request.urlopen(url)
  with open(outpath, "wb") as f:
    f.write(img.read())

Décomposons cette fonction. En argument on donne:
- un url vers l'image stocké sur internet (`url`)
- le chemin qui indique là où on voudrait le télécharger dans notre ordinateur (`outpath`).

D'abord, on crée une instance de l'image qui est stocké en ligne avec la fonction `urllib.request.urlopen()` du package `urllib`.

Puis on écrit le contenu comme un fichier dans notre ordinateur. Notez que nous avons utilisé le mot clé **with**. Il s'agit d'une forme plus succincte d'écrire ce que nous avons appris la dernière fois avec `open()`, `read()` et `close()`: en fait, on `open()` comme avant, mais on dit directement que la sortie de `open()` sera contneu dans `f`, et tous ce qui suit les deux points `:` sera effectué avec `f`. Quand on sort de cette indentation, `f` sera fermé automatiquement sans que nous ne soyons obligés de préciser avec `f.close()`.

Faisons un test:

In [ ]:
import os

# Je sais qu'une image se trouve accesible en direct ici:
image_url = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSF0Y2OY7alymw7VfJwVoxn0saaL6HIQE28zQ&s"

# Je définit là où je voudrais télécharger mon image:
chemin_de_sortie = os.path.join(os.getcwd(), "image_out.jpg")

# Je lance ma fonction:
download_image(image_url, chemin_de_sortie)

Désormais, on peut combiner cela avec notre code plus haut pour télécharger l'image principale d'un article aléatoire sur wikipedia:

In [ ]:
response = requests.get("https://fr.wikipedia.org/wiki/Special:Random")

if response.status_code == 200:
  print(response.url)
  bs = BeautifulSoup(response.content, 'html.parser')
  heading = bs.find(id = "firstHeading")

  # On recherche un element qui a la class "infobox"
  img_box = bs.find_all(class_="infobox")

  if len(img_box) > 0:
    # On recherche les elements img dans cet element:
    img_elements = img_box[0].find_all("img")

    if len(img_elements) > 0:
      # On ajoute https qui n'est pas renseigné par wikipedia:
      src_path = "https:" + img_elements[0].get("src")

      download_image(src_path, os.path.join(os.getcwd(), heading.string + ".jpg"))
    else:
      print("No images found")
  else:
    print("No info box found")

https://en.wikipedia.org/wiki/The_Simpsons


### Exercice

- Creer un script qui telecharge l'image principal d'une page wikipedia (si elle existe)
- Creer un script qui telecharge toutes les images d'une page wikipedia.
- Refaire ces execercises pour le site de votre choix.

## 3. APIs

Maintenant que l'on a vu cette première méthode de webscraping, passons à une solution qui est similaire: les APIs. Un API nous permet d'interagir avec un serveur à distance, et de manière beaucoup plus stable que le webscraping que nous avons vus jusqu'à lors. En essence, cel va fonctionne de la même manière que du webscraping classique: nous allons faire des requetes avec des urls. Mais au lieu de devoir fouiller de manière un peu bancale dans la reponse de notre requete avec quelque chose comme BeautifulSoup, on aura directement une réponse structuré.

Nous allons faire des tests aujours'hui avec l'API de Nakala. Vous pouvez trouver la documentation de l'API [ici](https://api.nakala.fr/doc), et aussi une instance de "test" que Nakala a mis en place pour pouvoir effectuer des tests [ici](https://test.nakala.fr/).

### Get

Dans un [RESTful API](https://en.wikipedia.org/wiki/REST) il y a 4 types d'opérations que nous pouvons faire:

- **get**: aller chercher des données existants.
- **post**: aller mettre des nouvelles données.
- **put**: modifier des données déjà existants.
- **delete**: supprimer des données existants.

Commencons avec **get**. Dans la documentation de l'API de Nakala, nous voyons qu'ils appelent un "document" une "data". Dans la documentation, on voit que pour aller trouver une data, ont doit suivre le syntaxe suivant: `/datas/{identifier}`. Faisons un exemple avec la data suivante qui est actuellement hébérgé chez Nakala: [https://www.nakala.fr/10.34847/nkl.def2v5a2](https://www.nakala.fr/10.34847/nkl.def2v5a2).

In [ ]:
import requests

nakala_api_prefix = "https://api.nakala.fr/"

rep = requests.get(nakala_api_prefix + "datas/10.34847/nkl.def2v5a2")

print(rep.status_code)
print(rep.json())

200
{'version': 1, 'collectionsIds': ['10.34847/nkl.d3563bvs'], 'files': [{'name': 'R_NCL_A16_09_02_40_00001.TIF', 'extension': 'TIF', 'size': '26234909', 'mime_type': 'image/tiff', 'sha1': 'b8809ba15fc42aecc6a19faab08cc77ce970c6bd', 'embargoed': '2019-01-16T00:00:00+01:00', 'description': None, 'humanReadableEmbargoedDelay': [], 'puid': 'fmt/353'}], 'lastModerator': None, 'lastModerationDate': None, 'relations': [], 'status': 'published', 'fileEmbargoed': False, 'uri': 'https://doi.org/10.34847/nkl.def2v5a2', 'identifier': '10.34847/nkl.def2v5a2', 'handleIdentifier': '11280/aba1615e', 'metas': [{'value': 'Carte n°35, terrain 45', 'lang': 'fr', 'typeUri': None, 'propertyUri': 'http://nakala.fr/terms#title'}, {'value': None, 'lang': None, 'typeUri': None, 'propertyUri': 'http://nakala.fr/terms#creator'}, {'value': None, 'lang': None, 'typeUri': None, 'propertyUri': 'http://nakala.fr/terms#created'}, {'value': 'CC-BY-NC-SA-4.0', 'lang': None, 'typeUri': None, 'propertyUri': 'http://nakal

In [ ]:
rep = requests.get(nakala_api_prefix + "search?q=map")

print(rep.status_code)
print(rep.json())

200
{'totalResults': 23307, 'datas': [{'status': 'public', 'uri': 'https://nakala.fr/collection/10.34847/nkl.569a75l1', 'identifier': '10.34847/nkl.569a75l1', 'metas': [{'value': 'Protocoles perpétuels', 'lang': 'fr', 'typeUri': None, 'propertyUri': 'http://nakala.fr/terms#title'}, {'value': 'Perpetual protocols', 'lang': 'en', 'typeUri': None, 'propertyUri': 'http://nakala.fr/terms#title'}, {'value': 'francophonie', 'lang': 'fr', 'typeUri': None, 'propertyUri': 'http://purl.org/dc/terms/subject'}, {'value': 'imaginaire social', 'lang': 'fr', 'typeUri': None, 'propertyUri': 'http://purl.org/dc/terms/subject'}, {'value': 'Nouvelle-Orléans, La (Etats-Unis, La.)', 'lang': 'fr', 'typeUri': None, 'propertyUri': 'http://purl.org/dc/terms/subject'}, {'value': 'recherche-création', 'lang': 'fr', 'typeUri': None, 'propertyUri': 'http://purl.org/dc/terms/subject'}, {'value': 'francophonie', 'lang': 'en', 'typeUri': None, 'propertyUri': 'http://purl.org/dc/terms/subject'}, {'value': 'social imagi